### Google Colab Configs

In [1]:
# Colab Config

%%capture
!pip install emoji
!pip install langdetect
!pip install nltk
!pip install tqdm
!pip install langid

from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


# Project

In [20]:
random_seed = 42
notebook_env = "test" # "test" for experimenting, "final" for final execution

In [6]:
import kagglehub
import os
import json
import pandas as pd
import spacy
import emoji
import re
from langdetect import detect, detect_langs
import nltk
import string
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from tqdm import tqdm
import langid
from ast import pattern
tqdm.pandas()

In [3]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### Download Data from Kaggle/Load Data

In [ ]:
# Download latest version
# path = kagglehub.dataset_download("jarredgaudineer/social-media-posts-fortune-1000-companies")

# print("Path to dataset files:", path)

In [ ]:
# Using the 'path' variable from your previous code
# downloaded_files = os.listdir(path)

# print("Downloaded files:", downloaded_files)

In [ ]:
# temp_df = pd.read_json(os.path.join(path, downloaded_files[0]))

In [ ]:
# temp_df['tweets'][0]

In [ ]:
# combine path with the file name
# files_directory = path
# path = os.path.join(files_directory, "10Mar2025.json")

In [11]:
# The List of Fortune 1000 Companies
# Download latest version
fort_list_path = kagglehub.dataset_download("jeannicolasduval/2024-fortune-1000-companies")

print("Path to dataset files:", fort_list_path)

for file in os.listdir(fort_list_path):
  print(file)

100%|██████████| 301k/301k [00:00<00:00, 78.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/jeannicolasduval/2024-fortune-1000-companies/versions/1
fortune1000_2024.csv
fortune1000_2024.parquet
fortune1000_companyinfo.parquet
fortune1000_companies.parquet


In [12]:
fortune_company_list = pd.read_csv(os.path.join(fort_list_path, "fortune1000_2024.csv"))
fortune_company_list.head()

,Rank,Company,Ticker,Sector,Industry,Profitable,Founder_is_CEO,FemaleCEO,Growth_in_Jobs,Change_in_Rank,...,Assets_M,CEO,Country,HeadquartersCity,HeadquartersState,Website,CompanyType,Footnote,MarketCap_Updated_M,Updated
0,1,Walmart,WMT,Retailing,General Merchandisers,yes,no,no,no,0.0,...,252399.0,C. Douglas McMillon,U.S.,Bentonville,Arkansas,https://www.stock.walmart.com,Public,"Figures are for fiscal year ended Jan. 31, 202...",559911.0,2024-08-05
1,2,Amazon,AMZN,Retailing,Internet Services and Retailing,yes,no,no,no,0.0,...,527854.0,Andrew R. Jassy,U.S.,Seattle,Washington,https://www.amazon.com,Public,"Market value as of July 15, 2024.",2005565.0,2024-08-05
2,3,Apple,AAPL,Technology,"Computers, Office Equipment",yes,no,no,no,1.0,...,352583.0,Timothy D. Cook,U.S.,Cupertino,California,https://www.apple.com,Public,"Figures are for fiscal year ended Sept. 30, 20...",3594309.0,2024-08-05
3,4,UnitedHealth Group,UNH,Health Care,Health Care: Insurance and Managed Care,yes,no,no,yes,1.0,...,273720.0,Andrew P. Witty,U.S.,Minnetonka,Minnesota,https://www.unitedhealthgroup.com,Public,"Market value as of July 15, 2024.",474339.0,2024-08-05
4,5,Berkshire Hathaway,BRKA,Financials,Insurance: Property and Casualty (Stock),yes,no,no,yes,2.0,...,1069978.0,Warren E. Buffett,U.S.,Omaha,Nebraska,https://www.berkshirehathaway.com,Public,"Market value as of July 15, 2024.",937028.0,2024-08-05


In [15]:
# Specify the file path within your Drive
# path = '/content/drive/MyDrive/companyDataSample.json'
path = '/content/drive/MyDrive/allCompanyData.json'

# Save the data as a JSON file
# with open(path, 'w') as f:
#     json.dump(data, f)

# print(f"Data saved to: {path}")

In [16]:
import json
with open(path, 'r') as f:
    data = json.load(f)

In [17]:
data[0].keys()

dict_keys(['company', 'rank', 'timestamp', 'tweets', 'posts', 'comments', 'metadata'])

In [18]:
columns = ["Company", "Text", "Source"]
atts_temp = ["tweets", "comments", "posts"]
source_dic = {"tweets": "Twitter", "comments": "Reddit-Comments", "posts": "Reddit-Posts"}
mp_doc_list = []

for company in data:
  for text_list in atts_temp:
    if text_list in company:
      for text in company[text_list]:
        mp_doc_list.append([company["company"], text["text"], source_dic[text_list]])


data_df = pd.DataFrame(mp_doc_list, columns=columns)
data_df

,Company,Text,Source
0,Walmart,$GOOGL $GOOG $WMT Google And Walmart Payment A...,Twitter
1,Walmart,Can $PHNIX become the first meme coin to dethr...,Twitter
2,Walmart,"Walmart $WMT \n\nThe Laughing Cow Cheese, 32 p...",Twitter
3,Walmart,Some of the most iconic businesses ever \n\n1....,Twitter
4,Walmart,Walmart $WMT currently has 1.05 Billion Square...,Twitter
...,...,...,...
591342,Clear Channel Outdoor Hldgs.,Hi BUB Sub!\n\n\nHad this account on the back ...,Reddit-Posts
591343,Clear Channel Outdoor Hldgs.,I am curious how these events typically play o...,Reddit-Posts
591344,Clear Channel Outdoor Hldgs.,,Reddit-Posts
591345,Clear Channel Outdoor Hldgs.,The Role of the National Weather Service in Em...,Reddit-Posts


In [21]:
enriched_data = pd.merge(fortune_company_list[["Company", "Ticker", "Sector", "Industry"]], data_df, left_on="Company", right_on="Company")
enriched_data.sample(10, random_state=random_seed)

,Company,Ticker,Sector,Industry,Text,Source
233120,BlackRock,BLK,Financials,Securities,You do understand that BlackRock buys the shar...,Reddit-Comments
420124,Hubbell,HUBB,Industrials,"Electronics, Electrical Equip.",,Reddit-Posts
221548,Adobe,ADBE,Technology,Computer Software,Your post is about Monkrus.\n\nMonkrus has no ...,Reddit-Comments
169118,Cummins,CMI,Industrials,Industrial Machinery,Hayden\n\nLanger\n\nPonting\n\nSmith\n\nClarke...,Reddit-Comments
88967,RTX,RTX,Aerospace & Defense,Aerospace & Defense,"You posted wrong links for stuff, double check...",Reddit-Comments
99040,HCA Healthcare,HCA,Health Care,Health Care: Medical Facilities,I work at an HCA hospital. We're a private EM ...,Reddit-Comments
469693,Splunk,NaN,Technology,Computer Software,"Over the past year, my org has moved from Splu...",Reddit-Posts
419354,Mattel,MAT,Household Products,"Toys, Sporting Goods",As Title!🙏\n\nPls genuine answer dena !!\n,Reddit-Posts
85860,Boeing,BA,Aerospace & Defense,Aerospace & Defense,These 757 planes and engines are all really ol...,Reddit-Comments
114387,Allstate,ALL,Financials,Insurance: Property and Casualty (Stock),The problem is exactly the people who consider...,Reddit-Comments


In [22]:
# compare unique companies in the enriched dataset and main dataset
enriched_companies = enriched_data["Company"].unique()
main_companies = data_df["Company"].unique()

print(f"Number of companies in the enriched dataset: {len(enriched_companies)}")
print(f"Number of companies in the main dataset: {len(main_companies)}")

Number of companies in the enriched dataset: 953
Number of companies in the main dataset: 953


In [23]:
data_df = enriched_data

## Preprocessing the texts

In [24]:
data_df.describe()

,Company,Ticker,Sector,Industry,Text,Source
count,591347,579881,591347,591347,591347,591347
unique,953,932,21,75,487445,3
top,Tesla,TSLA,Technology,Internet Services and Retailing,,Reddit-Posts
freq,9438,9438,132908,60577,82315,329493


In [25]:
# show duplicates
data_df[data_df.duplicated()].sample(10)

,Company,Ticker,Sector,Industry,Text,Source
175849,Netflix,NFLX,Media,Entertainment,,Reddit-Posts
511483,Cheesecake Factory,CAKE,"Hotels, Restaurants & Leisure",Food Services,"Alright, Naples foodies, I need your help! 🍝🍕\...",Reddit-Posts
561210,Roblox,RBLX,Media,Entertainment,,Reddit-Posts
575912,Griffon,GFF,Materials,"Building Materials, Glass",,Reddit-Posts
123060,Intel,INTC,Technology,Semiconductors and Other Electronic Components,,Reddit-Posts
288873,Ryder System,R,Transportation,Transportation and Logistics,,Reddit-Posts
337543,DoorDash,DASH,Technology,Internet Services and Retailing,A vintage inspired 3 piece suit I made for my ...,Reddit-Posts
329014,Oshkosh,OSK,Industrials,Construction and Farm Machinery,,Reddit-Posts
410305,Pool,POOL,Wholesalers,Wholesalers: Diversified,,Reddit-Posts
446995,Snap,SNAP,Technology,Internet Services and Retailing,,Reddit-Posts


In [26]:
# drop duplicates
data_df = data_df.drop_duplicates()
data_df.describe()

,Company,Ticker,Sector,Industry,Text,Source
count,490487,480398,490487,490487,490487,490487
unique,953,932,21,75,487445,3
top,Tesla,TSLA,Technology,Internet Services and Retailing,,Reddit-Comments
freq,7372,7372,110765,49625,883,242717


### Sample for Analysis
**Sampling of Enriched Data**: This is done to make experiments lighter, for final analysis all data will be used.

In [27]:
if notebook_env == "test":
  sample_data_df = data_df.sample(50000, random_state=random_seed)
elif notebook_env == "final":
  sample_data_df = data_df.copy()

sample_data_df.head()

,Company,Ticker,Sector,Industry,Text,Source
52944,Comcast,CMCSA,Telecommunications,Telecommunications,Same thing happened to me this week. Police De...,Reddit-Comments
143656,Dow,DOW,Chemicals,Chemicals,https://www.cnbc.com/2025/02/02/stock-market-t...,Reddit-Posts
525390,Sinclair,SBGI,Media,Entertainment,"Got some big names in the mid and fwd, feel my...",Reddit-Posts
374514,Olin,OLN,Chemicals,Chemicals,Not full true! I was playing with Olin earlier...,Reddit-Comments
195836,McDonald's,MCD,"Hotels, Restaurants & Leisure",Food Services,McDonald’s happened 2-3 times a week for many ...,Reddit-Comments


In [28]:
def preprocess_text(text, index, targets):

  # First Demojize Text
  edited_text = emoji.replace_emoji(text, '')

  # Mask company
  try:
    pattern = r'(?<![a-zA-Z])(?:' + '|'.join(re.escape(word) for word in targets if isinstance(word, str)) + r')(?![a-zA-Z])'
    if re.search(pattern, text, flags=re.IGNORECASE):
        edited_text = re.sub(pattern, "TargetedCompany", text, flags=re.IGNORECASE)
    else:
        edited_text = ""

  except Exception as e:
    raise

  # lower case
  edited_text = edited_text.lower()

  # remove links, mentions, hashtags
  edited_text = re.sub(r"http\S+|www\S+|@\w+|#\w+", '', edited_text)

  # Remove numbers
  edited_text = re.sub(r'\d+', '', edited_text)

  # Remove non ASCII
  edited_text = re.sub(r'[^\x00-\x7F]+', '', edited_text)

  # remove extra spaces
  edited_text = re.sub(r'\s+', ' ', edited_text).strip()

  # return if text is empty
  if edited_text == "":
    return edited_text

  # Remove non english
  try:
    # lang = detect(edited_text)
    lang, _ = langid.classify(edited_text)
    if lang != "en":
        edited_text = ""
  except Exception as e:
    # print(f"Error detecting language for text at index {index} and content:{edited_text}: {e}")
    edited_text = ""

  return edited_text


In [29]:
temp_df = sample_data_df.copy()
temp_df["Text"] = temp_df.progress_apply(lambda row: preprocess_text(row["Text"], row.name, [row["Company"], row["Ticker"]]), axis=1)
temp_df

100%|██████████| 50000/50000 [04:29<00:00, 185.19it/s]


,Company,Ticker,Sector,Industry,Text,Source
52944,Comcast,CMCSA,Telecommunications,Telecommunications,same thing happened to me this week. police de...,Reddit-Comments
143656,Dow,DOW,Chemicals,Chemicals,> stock futures tumbled sunday night to kick o...,Reddit-Posts
525390,Sinclair,SBGI,Media,Entertainment,"got some big names in the mid and fwd, feel my...",Reddit-Posts
374514,Olin,OLN,Chemicals,Chemicals,not full true! i was playing with targetedcomp...,Reddit-Comments
195836,McDonald's,MCD,"Hotels, Restaurants & Leisure",Food Services,,Reddit-Comments
...,...,...,...,...,...,...
390582,Huntsman,HUN,Chemicals,Chemicals,targetedcompany is a backline control hero on ...,Reddit-Posts
73167,Morgan Stanley,MS,Financials,Commercial Banks,,Reddit-Comments
49487,AT&T,T,Telecommunications,Telecommunications,,Reddit-Comments
170164,Bank of New York Mellon,BK,Financials,Commercial Banks,,Reddit-Posts


### Remove All blank

In [30]:
temp_df = temp_df[temp_df["Text"] != ""]
temp_df.describe()

,Company,Ticker,Sector,Industry,Text,Source
count,35192,34259,35192,35192,35192,35192
unique,821,808,21,75,34920,3
top,Tesla,AMZN,Technology,Internet Services and Retailing,targetedcompany,Reddit-Comments
freq,660,660,8823,4140,34,17724


### Wrap up pre-processing

In [31]:
temp_df.describe()

,Company,Ticker,Sector,Industry,Text,Source
count,35192,34259,35192,35192,35192,35192
unique,821,808,21,75,34920,3
top,Tesla,AMZN,Technology,Internet Services and Retailing,targetedcompany,Reddit-Comments
freq,660,660,8823,4140,34,17724


Saving the preprocesed data into the drive.

## Vectorization and Lemmatization

In [32]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
target_filter = "targetedcompany"

def tokenize_text(text):

  # first remove punct
  no_punct = [char for char in text if char not in string.punctuation]
  no_punct = "".join(no_punct)

  # replace ’ with '
  no_punct = no_punct.replace("’", "'")

  # Lemmatize
  doc = no_punct.split()

  tokens = []
  for token in doc:
    lemmed = lemmatizer.lemmatize(token)
    if lemmed not in stop_words and lemmed != target_filter:
      tokens.append(lemmed)

  return tokens

In [33]:
tokenized_texts = temp_df.sample(10, random_state=28).apply(lambda row: tokenize_text(row["Text"]), axis=1)

In [34]:
tokenized_texts

,0
52293,"[sorry, consistent, spam, regarding, posting, ..."
533059,"[typically, high, profile, wrestler, like, set..."
358942,"[didnt, use, bleach, reason, used, disinfectan..."
262641,"[check, rack, birkenstock, coupon, promo, code..."
454133,"[might, check, garden, beastly, delight, serie..."
121948,"[well, sound, like, acknowledgement, rather, m..."
52956,"[deadly, grasp, wa, many, many, year, strongly..."
322676,"[girlies, got, ticket, slc, around, orchestra,..."
505966,"[cant, believe, soninlaw, brought, brand, new,..."
294892,"[scoured, youtube, age, ended, using, adam, wi..."


In [35]:
vectorizer = CountVectorizer(analyzer=tokenize_text, min_df=0.005, max_df=0.995)
bow = vectorizer.fit_transform(temp_df["Text"])

In [36]:
len(vectorizer.vocabulary_)

1847

In [37]:
print(f"Shape of the BOW: {bow.shape}")
print(f"Number of non-zero values: {bow.nnz}")

Shape of the BOW: (35192, 1847)
Number of non-zero values: 1142541


In [38]:
tf_idf = TfidfTransformer()

In [39]:
tf_idf_text = tf_idf.fit_transform(bow)

In [40]:
tf_idf_text.shape

(35192, 1847)

## Topic Modeling The texts
Initially, we need to understand what sort of content we have. This will help deciding if we should go with data based topics or anchored topics.

In [41]:
lda = LatentDirichletAllocation(n_components=6, max_iter=10, learning_method='batch', random_state=42, verbose=1)

In [42]:
lda_op = lda.fit_transform(tf_idf_text)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [56]:
!pip install pyLDAvis==3.4.1
import pyLDAvis
import pyLDAvis.lda_model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.7 MB/s eta 0:00:00


In [57]:
# lda: your trained LatentDirichletAllocation model
    # bow: your CountVectorizer output (document-term matrix)
    # vectorizer: your CountVectorizer object

    # Prepare the visualization data
vis_data = pyLDAvis.lda_model.prepare(lda, bow, vectorizer)

In [58]:
# Display the interactive visualization
pyLDAvis.display(vis_data)

In [43]:
lda_op.shape

(35192, 6)

In [44]:
lda_op[0]

array([0.0428724 , 0.04252992, 0.04257798, 0.04264823, 0.78673862,
       0.04263285])

In [49]:
H1 = lda.components_

In [50]:
num_words = 20
vocab = np.array(vectorizer.get_feature_names_out())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [51]:
topics

['wa im like one time get dont would ha know love think feel really even day year thing also first',
 'market company stock inc share price ai k u data report growth location apply date ha january earnings description billion',
 'use game im x gb pc app using issue tv ha new code work like get would v need device',
 'wa im please link post anyone get shipping price looking order question one thanks would item seller ha hi new',
 'wa year get im account job month would pay work time ha dont got credit ive card like money day',
 'like dont would one im good get think make buy look thats people better ha know also really go even']

In [52]:
lda_op

array([[0.0428724 , 0.04252992, 0.04257798, 0.04264823, 0.78673862,
        0.04263285],
       [0.31346638, 0.58799934, 0.02463883, 0.02461388, 0.02462515,
        0.02465642],
       [0.77704346, 0.04440585, 0.04463654, 0.04455498, 0.04471042,
        0.04464875],
       ...,
       [0.4625636 , 0.02819443, 0.0282458 , 0.42446378, 0.02824965,
        0.02828274],
       [0.38625587, 0.15438705, 0.02562814, 0.02578112, 0.02600814,
        0.38193968],
       [0.04255693, 0.78700418, 0.04258835, 0.04265439, 0.04263707,
        0.04255908]])

In [53]:
temp_df["Topic"] = np.argmax(lda_op, axis=1)

<ipython-input-53-9a3cfce4063b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["Topic"] = np.argmax(lda_op, axis=1)


In [54]:
temp_df

,Company,Ticker,Sector,Industry,Text,Source,Topic
52944,Comcast,CMCSA,Telecommunications,Telecommunications,same thing happened to me this week. police de...,Reddit-Comments,4
143656,Dow,DOW,Chemicals,Chemicals,> stock futures tumbled sunday night to kick o...,Reddit-Posts,1
525390,Sinclair,SBGI,Media,Entertainment,"got some big names in the mid and fwd, feel my...",Reddit-Posts,0
374514,Olin,OLN,Chemicals,Chemicals,not full true! i was playing with targetedcomp...,Reddit-Comments,5
178759,3M,MMM,Chemicals,Chemicals,this storm had to be some kind of divine punis...,Reddit-Posts,0
...,...,...,...,...,...,...,...
374380,Olin,OLN,Chemicals,Chemicals,"gpa- fine, adequate for anywhere sat is on the...",Reddit-Comments,5
475737,Cadence,CDNS,Technology,Computer Software,is anyone else besides me sick and tired of th...,Reddit-Posts,0
330705,Seaboard,SEB,"Food, Beverages & Tobacco",Food Production,"hello r/trains, im doing a painting for th ann...",Reddit-Posts,0
390582,Huntsman,HUN,Chemicals,Chemicals,targetedcompany is a backline control hero on ...,Reddit-Posts,0


In [55]:
path = '/content/drive/MyDrive/allCompanyData_TopicModeled.csv'
# temp_df.to_csv(path)
# print(f"Data saved to: {path}")

In [ ]:
# Map topic index to a descriptive title
topic_titles = {
    0: "Casual Online Chatter",
    1: "Book or Media Reviews",
    2: "Consumer Goods / Retail",
    3: "Finance & Business News",
    4: "Job Applications & Recruitment",
    5: "Tech Support & Software Use"
}


In [ ]:
import matplotlib.pyplot as plt

# Count topics
topic_counts = temp_df["Topic"].value_counts().sort_index()

# Map the index to topic titles for labeling
labels = [topic_titles[i] for i in topic_counts.index]

# Plot with titles as x-axis labels
plt.figure(figsize=(10, 6))
plt.bar(labels, topic_counts.values)
plt.xticks(rotation=45, ha='right')
plt.title("Document Counts per Topic")
plt.ylabel("Number of Documents")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Group data by sector and topic, and count occurrences
sector_topic_counts = temp_df.groupby(['Sector', 'Topic'])['Topic'].count().unstack()

# Get sector and topic labels
sectors = sector_topic_counts.index
topic_labels = [topic_titles[i] for i in range(lda.n_components)]

# Create the stacked bar chart
fig, ax = plt.subplots(figsize=(12, 6))

# Plot bars for each topic within each sector
bottom = [0] * len(sectors)  # Initialize bottom for stacking
for i, topic_label in enumerate(topic_labels):
    ax.bar(sectors, sector_topic_counts[i], label=topic_label, bottom=bottom)
    bottom = [bottom[j] + sector_topic_counts[i][j] for j in range(len(sectors))]

# Set chart labels and formatting
ax.set_xlabel("Sector")
ax.set_ylabel("Number of Documents")
ax.set_title("Topic Distribution Across Sectors")
ax.legend(title="Topics", bbox_to_anchor=(1.05, 1), loc='upper left')  # Place legend outside the plot
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your DataFrame is named 'temp_df' and contains 'Source' and 'Topic' columns

# Group data by source and topic, and count occurrences
source_topic_counts = temp_df.groupby(['Source', 'Topic'])['Topic'].count().unstack()

# Get source and topic labels
sources = source_topic_counts.index
topic_labels = [topic_titles[i] for i in range(lda.n_components)]  # Assuming you have topic_titles defined

# Create the stacked bar chart
fig, ax = plt.subplots(figsize=(12, 6))

# Plot bars for each topic within each source
bottom = [0] * len(sources)  # Initialize bottom for stacking
for i, topic_label in enumerate(topic_labels):
    ax.bar(sources, source_topic_counts[i], label=topic_label, bottom=bottom)
    bottom = [bottom[j] + source_topic_counts[i][j] for j in range(len(sources))]

# Set chart labels and formatting
ax.set_xlabel("Source")
ax.set_ylabel("Number of Documents")
ax.set_title("Topic Distribution Across Sources")
ax.legend(title="Topics", bbox_to_anchor=(1.05, 1), loc='upper left')  # Place legend outside the plot
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()
plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Get feature names (words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Create a dictionary to store the word clouds for each topic
topic_wordclouds = {}

# Iterate through each topic
for topic_index in range(lda.n_components):
    # Get documents belonging to the current topic
    topic_documents = temp_df[temp_df["Topic"] == topic_index]["Text"]

    # Create a frequency dictionary for words in the topic documents
    word_frequencies = {}
    for doc in topic_documents:
        for word in tokenize_text(doc):  # Use your tokenize_text function
            if word in feature_names:
                word_frequencies[word] = word_frequencies.get(word, 0) + 1

    # Create a WordCloud object
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(word_frequencies)

    # Store the word cloud in the dictionary
    topic_wordclouds[topic_index] = wordcloud

    # Display the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud for Topic {topic_index}: {topic_titles[topic_index]}")  # Assuming you have topic_titles
    plt.show()


## Test extract ADJ

Text is there now, we need to extract the dimensions for each of the companies.

In [ ]:
dimensions = ["Effectiveness", "Innovation", "Ethics", "Empathy", "Aesthetic"]

In [ ]:
import spacy
from sentence_transformers import SentenceTransformer, util
from collections import defaultdict
import torch

# Load models
nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer("all-MiniLM-L6-v2")

# Define your semantic dimensions with positive/negative poles
dimensions = {
    "effectiveness": ("efficient", "inefficient"),
    "innovation": ("innovative", "outdated"),
    "trust": ("honest", "manipulative"),
    "empathy": ("caring", "cold"),
    "aesthetic": ("sleek", "ugly")
}

# Pre-encode the pole vectors for speed
anchor_vectors = {
    dim: {
        "pos": model.encode(pos, convert_to_tensor=True),
        "neg": model.encode(neg, convert_to_tensor=True)
    }
    for dim, (pos, neg) in dimensions.items()
}

In [ ]:
def score_adjective_on_dimensions(adj):
    vec = model.encode(adj, convert_to_tensor=True)
    scores = {}
    for dim, anchors in anchor_vectors.items():
        pos_sim = util.cos_sim(vec, anchors["pos"])
        neg_sim = util.cos_sim(vec, anchors["neg"])
        score = (pos_sim - neg_sim).item()
        scores[dim] = score
    return scores


In [ ]:
def extract_adjectives(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if token.pos_ == "ADJ"]


In [ ]:
company_profiles = defaultdict(lambda: defaultdict(list))

# for company, posts in data.items():
#     for post in posts:
#         adjectives = extract_adjectives(post)
#         for adj in adjectives:
#             dim_scores = score_adjective_on_dimensions(adj)
#             for dim, score in dim_scores.items():
#                 company_profiles[company][dim].append(score)

for company in preprocessed_text_data:
  for text in preprocessed_text_data[company]:
    adjectives = extract_adjectives(text)
    for adj in adjectives:
      dim_scores = score_adjective_on_dimensions(adj)
      for dim, score in dim_scores.items():
        company_profiles[company][dim].append(score)


KeyboardInterrupt: 

In [ ]:
import numpy as np

final_scores = {}
for company, dims in company_profiles.items():
    final_scores[company] = {
        dim: round(np.mean(scores), 3)
        for dim, scores in dims.items() if scores
    }

from pprint import pprint
pprint(final_scores)


In [ ]:
# # 5 random items from list
# import random
# sample = random.sample(preprocessed_text_data["Walmart"], 10)

# for samp in sample:
#   print("============================ THE TEXT IS =============================")
#   print(samp)
#   print("                            THE ADJECTIVES ARE                        ")
#   print(extract_adjectives(samp))
#   print("==================================================================")

In [ ]:
preprocessed_text_data["Walmart"][2000]

## Sentence based

In [ ]:
company_profiles_wholeText = defaultdict(lambda: defaultdict(list))

for company in preprocessed_text_data:
  for text in preprocessed_text_data[company]:
    dim_scores = score_adjective_on_dimensions(text)
    for dim, score in dim_scores.items():
      company_profiles_wholeText[company][dim].append(score)

In [ ]:
# dimensions of a dictionary
company_profiles_wholeText.keys()

In [ ]:

final_scores_wholeText = {}
for company, dims in company_profiles_wholeText.items():
    final_scores_wholeText[company] = {
        dim: round(np.mean(scores), 3)
        for dim, scores in dims.items() if scores
    }

from pprint import pprint
pprint(final_scores_wholeText)
